In [1]:
dataset_id = "imdb_modeled"
!bq --location=US mk --dataset {dataset_id} 

BigQuery error in mk operation: Dataset 'starry-center-266501:imdb_modeled'
already exists.


## Create new table "People" from columns of imdb_staging.NameBasics. Establish nConst as a primary key. Retitle "string_field_i" as correct name. Cast birthYear and deathYear from string to int

In [1]:
%%bigquery
create table imdb_modeled.People as 
SELECT distinct string_field_0 as nConst, string_field_1 as primaryName, safe_cast(string_field_2 as int64) as birthYear,
safe_cast(string_field_3 as int64) as deathYear, string_field_4 as primaryProfession, string_field_5 as knownForTitles
FROM imdb_staging.NameBasics

""


### Sample People

In [16]:
%%bigquery 
SELECT * from imdb_modeled.People limit 5

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm1072898,Hans-Joachim Frank,1954,NaN,"actor,director,assistant_director","tt1091772,tt0078264,tt0302888,tt1191976"
1,nm0074890,Bärbl Bergmann,1931,2003.0,"director,writer,assistant_director","tt0130988,tt0310958,tt1298811,tt0085199"
2,nm2228559,Juan Souter,1970,NaN,"art_department,set_decorator","tt2980592,tt1263670,tt0406759,tt0806203"
3,nm0392646,Jes Holtsø,1956,NaN,"actor,art_department,composer","tt0075002,tt0073482,tt0066807,tt0068081"
4,nm0884620,Frantisek Valert,1932,1993.0,"cinematographer,camera_department,special_effects","tt0058971,tt0062332,tt0232750,tt0405822"


### Verify nConst as a primary key

In [15]:
%%bigquery
SELECT COUNT(*) FROM imdb_modeled.People

,f0_
0,9870691


In [22]:
%%bigquery
SELECT COUNT(DISTINCT nConst) FROM imdb_modeled.People

,f0_
0,9870691


## Create table "Titles" from columns of imdb_staging.TitleBasics. Establish primary key tConst. Cast strings endYear and runtimeMinutes to integers

In [27]:
%%bigquery
CREATE TABLE imdb_modeled.Titles as 
SELECT DISTINCT tConst, titleType, primaryTitle, startYear, safe_cast(endYear as int64) as endYear, 
safe_cast(runtimeMinutes as int64) as runtimeMinutes
FROM imdb_staging.TitleBasics

""


### Sample Titles

In [28]:
%%bigquery
SELECT * from imdb_modeled.Titles 
LIMIT 5

,tConst,titleType,primaryTitle,startYear,endYear,runtimeMinutes
0,tt10894890,tvEpisode,Episode dated 2 March 1999,1999,None,NaN
1,tt0607122,tvEpisode,Episode dated 29 November 1999,1999,None,NaN
2,tt0488234,short,Tucumán arde: documento,1999,None,NaN
3,tt2096773,tvEpisode,Episode #1.76,1999,None,NaN
4,tt10710646,tvEpisode,Taose! Kuraken,1999,None,20.0


### Verify tConst as a primary key 

In [29]:
%%bigquery
SELECT COUNT(*) from imdb_modeled.Titles

,f0_
0,6524093


In [33]:
%%bigquery
SELECT COUNT (DISTINCT tConst)from imdb_modeled.Titles

,f0_
0,6524093


## Create table "Writes" from writers and tConst columns of imdb_staging.TitleCrew. Establish primary key tConst. Use parent table Titles to place foreign key restriction on Writes for the tConst field. Name unnamed fields of TitleCrew.

In [15]:
%%bigquery
create table imdb_modeled.Writes as 
SELECT DISTINCT tc.string_field_2 as writers, tc.string_field_0 as tConst
FROM imdb_staging.TitleCrew tc
INNER JOIN imdb_modeled.Titles ti on tc.string_field_0 = ti.tConst

""


### Verify tConst as a primary key of Writes

In [16]:
%%bigquery
SELECT COUNT (*) 
FROM imdb_modeled.Writes

,f0_
0,6524093


In [17]:
%%bigquery
SELECT COUNT (DISTINCT tConst)
FROM imdb_modeled.Writes

,f0_
0,6524093


### Verify imdb_modeled.Writes has a foreign key constraint on field tConst by parent imdb_modeled.Titles

In [18]:
%%bigquery
SELECT wi.tConst
FROM imdb_modeled.Writes wi
WHERE wi.tConst NOT IN (SELECT ti.tConst FROM imdb_modeled.Titles ti)

,tConst


## Create table "Directs" from directors and tConst columns of imdb_staging.TitleCrew. Create primary key of tConst. Place foreign key restriction on Directors for the tConst field of parent table imdb_modeled.Titles. Name unnamed fields of TitleCrew.

In [19]:
%%bigquery
CREATE table imdb_modeled.Directs as 
SELECT DISTINCT tc.string_field_1 as directors, tc.string_field_0 as tConst
FROM imdb_staging.TitleCrew tc
INNER JOIN imdb_modeled.Titles ti on tc.string_field_0 = ti.tConst

""


### Sample Directs

In [20]:
%%bigquery
SELECT * 
FROM imdb_modeled.Directs
LIMIT 5

,directors,tConst
0,nm0299154,tt0011983
1,nm0510357,tt0036917
2,nm0742819,tt0082495
3,nm0815638,tt0286200
4,nm1143942,tt0326989


## Verify tConst as a primary key 

In [21]:
%%bigquery
SELECT COUNT(*) FROM imdb_modeled.Directs

,f0_
0,6524093


In [22]:
%%bigquery
SELECT COUNT(DISTINCT tConst) FROM imdb_modeled.Directs

,f0_
0,6524093


### Verify imdb_modeled.Direct has a foreign key constraint on field tConst by parent imdb_modeled.Titles

In [23]:
%%bigquery
SELECT di.tConst
FROM imdb_modeled.Directs di
WHERE di.tConst NOT IN (SELECT ti.tConst FROM imdb_modeled.Titles ti)

,tConst


## Create table "TitleRatings" from imdb_modeled.TitleRatings, with primary key tConst and foreign key constraint on tConst by parent table Titles

In [54]:
%%bigquery
CREATE table imdb_modeled.TitleRatings as
SELECT distinct tr.tConst, averageRating, numVotes
FROM imdb_staging.TitleRatings tr
INNER JOIN imdb_staging.TitleBasics tb on tb.tConst = tr.tConst

""


## Sample TitleRatings

In [55]:
%%bigquery
SELECT * 
FROM imdb_modeled.TitleRatings 
LIMIT 5

,tConst,averageRating,numVotes
0,tt0049468,4.1,319
1,tt0077533,4.2,5864
2,tt0050514,4.1,38
3,tt0077620,4.2,212
4,tt0064307,3.2,607


### Verify tConst as a primary key

In [56]:
%%bigquery
SELECT COUNT(*) 
FROM imdb_modeled.TitleRatings

,f0_
0,1022151


In [57]:
%%bigquery
SELECT COUNT(DISTINCT tConst)
FROM imdb_modeled.TitleRatings

,f0_
0,1022151


### Verify imdb_modeled.TitleRatings has a foreign key constraint on field tConst by parent imdb_modeled.Titles

In [2]:
%%bigquery
SELECT tr.tConst
FROM imdb_modeled.TitleRatings tr
WHERE tr.tConst NOT IN (SELECT ti.tConst FROM imdb_modeled.Titles ti)

,tConst


## Create table Characters with fields from imdb_staging.TitlePrincipals with primary key tConst, and foreign key restraint on field nConst by parent table People

In [58]:
%%bigquery
CREATE table imdb_modeled.Characters as
SELECT DISTINCT tConst, tp.nConst, job, characters
FROM imdb_staging.TitlePrincipals tp
INNER JOIN imdb_modeled.People pe 
ON pe.nConst = tp.nConst

""


## Sample Characters

In [59]:
%%bigquery 
SELECT *
FROM imdb_modeled.Characters
LIMIT 5

,tConst,nConst,job,characters
0,tt4533768,nm0233893,\N,"[""Tilly Winn""]"
1,tt7328956,nm2362231,adaptation,\N
2,tt4844918,nm6056794,\N,\N
3,tt10514508,nm10779526,\N,\N
4,tt10987232,nm0616338,screenplay,\N


### Verify tConst as a primary key

In [60]:
%%bigquery
SELECT COUNT (*)
FROM imdb_modeled.Characters

,f0_
0,37687123


In [61]:
%%bigquery
SELECT COUNT (DISTINCT tConst) FROM imdb_modeled.Characters

,f0_
0,5899527


### Verify foreign key constraint on nConst by parent table People 

In [1]:
%%bigquery
SELECT ch.nConst
FROM imdb_modeled.Characters ch
WHERE ch.nConst NOT IN (SELECT pe.nConst FROM imdb_modeled.People pe)

,nConst


In [11]:
# Using Apache Beam, create a table Directs_Beam, that makes Directs 3NF Compliant 
# Changes nConst field from an array of directors to a single primary director 

%run Directs_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'imdb_modeled'
 projectId: 'starry-center-266501'
 tableId: 'Directs'> referenced by query SELECT directors, tConst FROM imdb_modeled.Directs limit 100
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.10 seconds.
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.10 seconds.
I

# Verify that Directs_Beam has a primary key tConst

In [8]:
%%bigquery
select count(*) from imdb_modeled.Directs_Beam

,f0_
0,100


In [9]:
%%bigquery
select count(distinct tConst) from imdb_modeled.Directs_Beam

,f0_
0,100


# Verify that Directs_Beam has a foreign key constraint placed on it by Titles

In [10]:
%%bigquery 
SELECT db.tConst
FROM imdb_modeled.Directs_Beam db
WHERE db.tConst NOT IN (SELECT t.tConst FROM imdb_modeled.Titles t)

,tConst
